In [3]:
#import dependencies 
import pandas as pd 
import requests 
import json 
import snowflake.connector
import config 
import os 
import time 

In [2]:
#cereate connection to specigic database/schema 
con = snowflake.connector.connect(
    user=config.keys['user'],
    password=config.keys['password'],
    account=config.keys['account'],
    warehouse = 'WAREHOUSE_BLUE',
    database  = 'DATABASE_BLUE',
    schema = 'SCHEMA_BLUE'
)

In [4]:
create_table_query = (
    """create or replace TABLE DATABASE_BLUE.SCHEMA_BLUE.BLUE4 (
	"fiscal_year" VARCHAR(50),
	"payroll_number" VARCHAR(50),
	"agency_name" VARCHAR(50),
	"last_name" VARCHAR(50),
	"first_name" VARCHAR(50),
	"mid_init" VARCHAR(50),
	"agency_start_date" VARCHAR(50),
	"work_location_borough" VARCHAR(50),
	"title_description" VARCHAR(50),
	"leave_status_as_of_june_30" VARCHAR(50),
	"base_salary" VARCHAR(50),
	"pay_basis" VARCHAR(50),
	"regular_hours" VARCHAR(50),
	"regular_gross_paid" VARCHAR(50),
	"ot_hours" VARCHAR(50),
	"total_ot_paid" VARCHAR(50),
	"total_other_pay" VARCHAR(50)
);""" 
) 

# execute
con.cursor().execute(create_table_query)

In [5]:
#get dataset info for funciton 
#get row count for dataset 
data_sample = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json?$select=count(*)")
data_sample.json()
data_size = data_sample.json()[0]['count']
sample_df = pd.DataFrame(data_sample)

In [13]:
data_sample.headers

{'Server': 'nginx', 'Date': 'Sun, 24 Dec 2023 19:58:28 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'ETag': '"YWxwaGEuNzQzMjJfNV8xNTN2TE4tc09USlAzbWFsZV9IdGtSckJEMnM4OEE--gziB6woCHGSBIxSMEUEpO0mokJWAVY---gzip"', 'X-SODA2-Fields': '["count"]', 'X-SODA2-Types': '["number"]', 'X-SODA2-Data-Out-Of-Date': 'false', 'X-SODA2-Truth-Last-Modified': 'Tue, 28 Nov 2023 17:52:54 GMT', 'X-SODA2-Secondary-Last-Modified': 'Tue, 28 Nov 2023 17:52:54 GMT', 'Last-Modified': 'Tue, 28 Nov 2023 17:52:54 GMT', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Age': '0', 'X-Socrata-Region': 'aws-us-east-1-fedramp-prod', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Socrata-RequestId': '731331d0e88656c29bc7928849b82c6d'}

In [ ]:
#get years in dataset
years = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json?$select=distinct fiscal_year")
years = years.json()
years = [i['fiscal_year'] for i in years]
years.sort()

In [67]:
data_size

'5662713'

In [77]:
response.status_code

200

In [83]:
print(3)

3


In [125]:
for i in 'simp':
    
    print(i)

s
i
m
p


In [127]:
for i in years: 
    #define function variables 
    url = f"https://data.cityofnewyork.us/resource/k397-673e.json?$limit={300}&$where=fiscal_year={i}"
    #using dataset size as arbitrary limit
    response = requests.get(url)
    
    if response.status_code != 200:
        break 
        
    file_path = f'temp/{i}_data_sample.csv'
    #save data to temp folder as csv    
    dataframe = pd.DataFrame(response.json())
    dataframe.to_csv(file_path, index=False)   
    
    print(len(dataframe.columns))
    print(f'{i}:rows:{len(dataframe)}')
    
    #stage file in snowflake 
    #con.cursor().execute(f'PUT file://{file_path}* @%BLUE4')
    
    #free up memory, clear out variable and delete temp file 
    dataframe = None
    #if os.path.isfile(file_path):
     #   os.remove(file_path)
    
    #
    #time.sleep(10)

16
2014:rows:300
16
2015:rows:300
16
2016:rows:300
16
2017:rows:300
17
2018:rows:300
17
2019:rows:300
17
2020:rows:300
17
2021:rows:300
17
2022:rows:300
17
2023:rows:300


In [130]:
- column number changes 
need to have that column ready to be set as null when no inserts on it 
- need to reorder dataframes before saving to make sure order matches table, AND add blank column if does not exist so it matches fully ?


SyntaxError: invalid syntax (<ipython-input-130-3e06f7808ddd>, line 1)

In [117]:
df16 = pd.read_csv('temp/2015_data_sample.csv')

In [114]:
df17 = pd.read_csv('temp/2017_data_sample.csv')

In [128]:
len(df16.columns.to_list())

16

In [129]:
len(df17.columns.to_list())

16

In [105]:
os.listdir('temp/')

[]

In [107]:
#include "/" at the end of dir path 
def clear_dir(dir_path): 
    for i in os.listdir(dir_path):
        os.remove(f"{dir_path}{i}")
    

In [111]:
clear_dir('temp/')